In [ ]:
import cv2
import numpy as np
from datascience import *
import matplotlib as mpl
from matplotlib import pyplot as plt
import sys
import os
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 150

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

In [ ]:
images = []
image_names = []
labeled_imgs = []
original_img1 = []
iteratiton_amnt = []

original_img = []
filter_img = []

In [ ]:
#access img directory
dirname = 'Anolis_cristatellus_images/'

#create an iterator object for img directory
directory = os.scandir(dirname)

print(os.listdir(dirname))


images = []
block_and_iteration = []

def initial_binarization(initial_block_size, initial_iteration, customized):
    index = 0;
    images.clear
    
    #iterates through each image in the directory
    for img in directory:
        img_name = img.name
        if (img_name != '.ipynb_checkpoints'):
            
            image_names.append(img.name)
            
            #read the image, convert to grayscale add to array and title + blur
            img = cv2.imread(dirname + "" + img_name)
            original_img1.append(img)
            
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            original_img.append(gray)
            title = img_name
            img = gray
            blur = cv2.GaussianBlur(img,(5,5),0)
            blur = cv2.bilateralFilter(blur,10,10,100)
            
            filter_img.append(blur)
            
            #if the customized block size has not yet been determined this if statement will run
            if(len(customized)==0):
                ret, thresh_Otsu = cv2.threshold(blur,120 , 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                thresh_high_iteration = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size[index][0], -2)
            
            else:
                print("Customized Values used")
                print(block_size[index][0], block_size[index][1])
                thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, block_size[index][0], -2)
                
            kernel = np.ones((3,3),np.uint8)
            
            #if the customized iteration number has not yet been determined this if statement will run
            if(len(customized)==0):
                opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 4)
                opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 4)

            
            #if customized array is greater than size 0 iteration number has been determined
            else:
                opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations=block_size[index][1])

            
            # sure background area
            sure_bg = cv2.dilate(opening,kernel,iterations=1)

            # Finding sure foreground area: What does cv2.DIST do?
            dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
            ret, sure_fg = cv2.threshold(dist_transform, 1.5, 255, 0) 

            # Finding unknown region
            sure_fg = np.uint8(sure_fg)
            unknown = cv2.subtract(sure_bg,sure_fg)
            
            #adding sure_fg binarization to list of images
            images.append(sure_fg)

            plt.subplot(1,3,1), plt.imshow(gray, 'gray')
            plt.subplot(1,3,2), plt.imshow(sure_fg, 'gray')
            
            
            # Label and count (taken from maggie's code)
            # source: https://medium.com/analytics-vidhya/images-processing-segmentation-and-objects-counting-in-an-image-with-python-and-opencv-216cd38aca8e
            count, labels = cv2.connectedComponents(sure_fg)                
            count = count - 1 # background doesn't count as a scale
            label_hue = np.uint8(179 * labels / np.max(labels))
            blank_ch = 255 * np.ones_like(label_hue)
            labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])
            labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
            labeled_img[label_hue == 0] = 0
            
            if(len(customized) != 0):
                labeled_imgs.append(labeled_img)

            #plt.subplot(1, 3, 3), plt.imshow(labeled_img, 'gray')
            #plt.title('Count: ' + str(count))
            #plt.show()
            index += 1

print("INITIAL TEST BINARIZATION\n")
initial_binarization(41, 4, [])
